In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from efficientnet_pytorch import EfficientNet
from torch.utils.data import DataLoader

# Путь к папкам с данными
data_dir = 'C:\\Users\\pelevinmd\\Desktop\\data'

# Размер батча и количество эпох
batch_size = 4
num_epochs = 5

# Преобразования изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация средних значений и стандартных отклонений для ImageNet
])

In [6]:
# Загрузка тренировочного и тестового датасетов
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'cars_train'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'cars_test'), transform=transform)

# Создание загрузчиков данных
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Инициализация модели EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=196)  # Загрузка предварительно обученных весов для EfficientNet-B0

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(model.parameters())

# Перемещение модели на графический процессор, если доступен
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


Loaded pretrained weights for efficientnet-b0


In [7]:
# Обучение модели
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_loss /= len(train_loader)
    train_accuracy = correct / total
    
    # Вычисление точности на тестовом датасете после каждой эпохи
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_loss /= len(test_loader)
    test_accuracy = correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f} - Train Accuracy: {100*train_accuracy:.2f}% - Test Loss: {test_loss:.4f} - Test Accuracy: {100*test_accuracy:.2f}%')

Epoch [1/5] - Train Loss: 0.0135 - Train Accuracy: 99.93% - Test Loss: 0.0032 - Test Accuracy: 99.98%
Epoch [2/5] - Train Loss: 0.0000 - Train Accuracy: 100.00% - Test Loss: 0.0016 - Test Accuracy: 99.99%
Epoch [3/5] - Train Loss: 0.0000 - Train Accuracy: 100.00% - Test Loss: 0.0025 - Test Accuracy: 99.99%
Epoch [4/5] - Train Loss: 0.0000 - Train Accuracy: 100.00% - Test Loss: 0.0005 - Test Accuracy: 100.00%
Epoch [5/5] - Train Loss: 0.0000 - Train Accuracy: 100.00% - Test Loss: 0.0019 - Test Accuracy: 99.98%
